In [1]:
!pip install pandastable

  Preparing metadata (setup.py): started
  Preparing metadata (setup.py): finished with status 'done'
     ---------------------------------------- 0.0/717.0 kB ? eta -:--:--
     --------------------------- ---------- 524.3/717.0 kB 3.4 MB/s eta 0:00:01
     -------------------------------------- 717.0/717.0 kB 3.6 MB/s eta 0:00:00
  Preparing metadata (setup.py): started
  Preparing metadata (setup.py): finished with status 'done'
   ---------------------------------------- 0.0/721.5 kB ? eta -:--:--
   --------------------------------------- 721.5/721.5 kB 10.0 MB/s eta 0:00:00
  Created wheel for pandastable: filename=pandastable-0.14.0-py3-none-any.whl size=257404 sha256=81e852615ef86e33006c0d84332615006e20c7e96abcde2402b3f07dddcd7f7b
  Stored in directory: c:\users\dell\appdata\local\pip\cache\wheels\81\59\85\c7b297bbe2b022c6012035f97d339badbe005f57706d635024
  Created wheel for odfpy: filename=odfpy-1.4.1-py2.py3-none-any.whl size=137512 sha256=0824c026f9570f976931bafa37c9bee975

In [9]:
! pip install tkcalendar

In [ ]:
import tkinter as tk
from tkinter import messagebox, Toplevel, Scrollbar # Added Scrollbar
from tkcalendar import Calendar # Requires: pip install tkcalendar
from datetime import datetime
import json # Added explicit import for goal handling
import csv
import io
import os # Needed for file deletion during reset
from collections import defaultdict

# --- GLOBAL DATA STRUCTURES ---
FINANCE_GOALS = {
    'monthly_salary': 0.0,
    'saving_goal': 0.0,
    'max_spending_budget': 0.0
}
EXPENSES = []
# File paths
DATA_FILE = 'budget_data.csv'  # Expenses file
GOAL_FILE = 'budget_goals.json' # Goals file

# --- FILE OPERATIONS (Robust CSV Handling) ---
def save_goals():
    """Saves the current goals to a JSON file."""
    try:
        with open(GOAL_FILE, 'w') as f:
            json.dump(FINANCE_GOALS, f, indent=4)
    except Exception as e:
        print(f"Error saving goals: {e}")

def load_goals():
    """Loads goals from the JSON file."""
    try:
        with open(GOAL_FILE, 'r') as f:
            data = json.load(f)
            FINANCE_GOALS.update(data)
    except (FileNotFoundError, json.JSONDecodeError):
        pass # Start with defaults if file is missing or corrupt

def save_expenses_csv():
    """Saves the current expenses to a CSV file, ensuring all fields are present."""
    # Sort by date before saving for better readability
    EXPENSES.sort(key=lambda x: datetime.strptime(x['date'], '%Y-%m-%d'), reverse=True)

    fieldnames = ['date', 'category', 'amount']

    # Sanitize data to ensure all keys exist and amount is a string
    sanitized_expenses = []
    for expense in EXPENSES:
        sanitized_row = {
            'date': expense.get('date', datetime.now().strftime('%Y-%m-%d')),
            'category': expense.get('category', 'Uncategorized'),
            # Ensure amount is correctly formatted to 2 decimal places when saving
            'amount': f"{expense.get('amount', 0.0):.2f}"
        }
        sanitized_expenses.append(sanitized_row)

    try:
        with open(DATA_FILE, 'w', newline='', encoding='utf-8') as csvfile:
            writer = csv.DictWriter(csvfile, fieldnames=fieldnames)
            writer.writeheader()
            writer.writerows(sanitized_expenses)
    except Exception as e:
        print(f"❌ Error saving expenses to CSV: {e}")

def load_expenses_csv():
    """Loads expenses from the CSV file."""
    EXPENSES.clear()
    try:
        with open(DATA_FILE, 'r', encoding='utf-8') as csvfile:
            reader = csv.DictReader(csvfile)
            for row in reader:
                try:
                    # Type conversion/cleaning for consistency
                    row['amount'] = float(row['amount'])
                    EXPENSES.append(row)
                except ValueError:
                    print(f"Skipping malformed row in CSV: {row}")
    except FileNotFoundError:
        pass # Start fresh if file doesn't exist

# --- GUI CLASS ---

class BudgetApp:
    def __init__(self, master):
        self.master = master
        master.title("💰 Pay Yourself First Budget Tracker")
        master.configure(bg="#f4f4f4")

        load_goals()
        load_expenses_csv()

        # Tkinter variables to hold input values
        self.salary_var = tk.DoubleVar(value=FINANCE_GOALS['monthly_salary'])
        self.saving_var = tk.DoubleVar(value=FINANCE_GOALS['saving_goal'])
        self.exp_amount_var = tk.DoubleVar()
        self.exp_category_var = tk.StringVar()
        self.exp_date_var = tk.StringVar(value=datetime.now().strftime('%Y-%m-%d'))

        # --- Create Main Frames ---
        self.goal_frame = tk.LabelFrame(master, text="💵 Monthly Goals & Control", padx=15, pady=15, bg="#fff", font=('Arial', 10, 'bold'))
        # Expense frame now holds input + list
        self.expense_frame = tk.LabelFrame(master, text="💸 Expense Management", padx=15, pady=15, bg="#fff", font=('Arial', 10, 'bold'))
        self.summary_frame = tk.LabelFrame(master, text="📊 Current Month Status", padx=15, pady=15, bg="#fff", font=('Arial', 10, 'bold'))
        self.conclusion_frame = tk.LabelFrame(master, text="✨ Overall Conclusion", padx=15, pady=15, bg="#fff", font=('Arial', 10, 'bold'))

        self.goal_frame.grid(row=0, column=0, padx=10, pady=5, sticky="ew")
        self.expense_frame.grid(row=1, column=0, padx=10, pady=5, sticky="nsew") # Allows it to grow
        self.summary_frame.grid(row=0, column=1, padx=10, pady=5, sticky="nsew")
        self.conclusion_frame.grid(row=1, column=1, padx=10, pady=5, sticky="nsew")

        # Configure grid weights to allow resizing
        master.grid_columnconfigure(0, weight=1)
        master.grid_columnconfigure(1, weight=1)
        master.grid_rowconfigure(0, weight=0) # Goals frame doesn't need to grow much
        master.grid_rowconfigure(1, weight=2) # Expense frame + List and summaries should grow

        # --- Goal Frame Widgets ---
        tk.Label(self.goal_frame, text="Monthly Salary:", bg="#fff").grid(row=0, column=0, sticky="w", pady=5)
        tk.Entry(self.goal_frame, textvariable=self.salary_var, width=20).grid(row=0, column=1, padx=5, pady=5)

        tk.Label(self.goal_frame, text="Saving Goal:", bg="#fff").grid(row=1, column=0, sticky="w", pady=5)
        tk.Entry(self.goal_frame, textvariable=self.saving_var, width=20).grid(row=1, column=1, padx=5, pady=5)

        tk.Button(self.goal_frame, text="Set Goals", command=self.set_goals_gui, bg="#4CAF50", fg="white").grid(row=2, column=0, columnspan=2, pady=(10, 5), sticky='ew')
        
        # --- NEW RESET BUTTON ---
        tk.Button(self.goal_frame, text="🗑️ Reset ALL Data", command=self.reset_all_data, bg="#f44336", fg="white").grid(row=3, column=0, columnspan=2, pady=(5, 0), sticky='ew')


        # --- Expense Input Widgets (inside expense_frame, top section) ---
        input_frame = tk.Frame(self.expense_frame, bg="#fff")
        input_frame.grid(row=0, column=0, sticky='ew', columnspan=3)
        input_frame.grid_columnconfigure(1, weight=1) # Allow input fields to stretch

        tk.Label(input_frame, text="Date (YYYY-MM-DD):", bg="#fff").grid(row=0, column=0, sticky="w", pady=5, padx=5)
        date_entry = tk.Entry(input_frame, textvariable=self.exp_date_var, width=15, state='readonly')
        date_entry.grid(row=0, column=1, padx=(5,0), pady=5, sticky='w')
        tk.Button(input_frame, text="📅", command=self.open_calendar, width=2).grid(row=0, column=2, padx=(0, 5), pady=5, sticky='w')

        tk.Label(input_frame, text="Amount:", bg="#fff").grid(row=1, column=0, sticky="w", pady=5, padx=5)
        tk.Entry(input_frame, textvariable=self.exp_amount_var, width=20).grid(row=1, column=1, columnspan=2, padx=5, pady=5, sticky='ew')

        tk.Label(input_frame, text="Category:", bg="#fff").grid(row=2, column=0, sticky="w", pady=5, padx=5)
        tk.Entry(input_frame, textvariable=self.exp_category_var, width=20).grid(row=2, column=1, columnspan=2, padx=5, pady=5, sticky='ew')

        tk.Button(input_frame, text="➕ Add Expense", command=self.add_expense_gui, bg="#008CBA", fg="white").grid(row=3, column=0, columnspan=3, pady=10, sticky='ew', padx=5)

        # --- Expense List (inside expense_frame, bottom section) ---
        tk.Label(self.expense_frame, text="--- Logged Expenses (Select to Delete) ---", bg="#fff", font=('Arial', 9, 'italic')).grid(row=1, column=0, sticky='ew', pady=(10, 0))

        list_frame = tk.Frame(self.expense_frame)
        list_frame.grid(row=2, column=0, sticky="nsew", padx=5, pady=5)
        self.expense_frame.grid_rowconfigure(2, weight=1) # Listbox should take up vertical space
        self.expense_frame.grid_columnconfigure(0, weight=1)

        self.list_scrollbar = Scrollbar(list_frame, orient=tk.VERTICAL)
        # Monospaced font helps with alignment
        self.expense_listbox = tk.Listbox(list_frame, yscrollcommand=self.list_scrollbar.set, height=10, font=('Courier', 9))
        self.list_scrollbar.config(command=self.expense_listbox.yview)

        self.list_scrollbar.pack(side=tk.RIGHT, fill=tk.Y)
        self.expense_listbox.pack(side=tk.LEFT, fill=tk.BOTH, expand=True)

        tk.Button(self.expense_frame, text="❌ Delete Selected Expense", command=self.delete_expense_gui, bg="#FF6347", fg="white").grid(row=3, column=0, pady=5, sticky='ew', padx=5)

        # --- Summary Frame (Current Month) ---
        self.summary_text = tk.Text(self.summary_frame, height=15, width=40, state=tk.DISABLED, bg="#f9f9f9", font=('Courier', 9))
        self.summary_text.grid(row=0, column=0, padx=5, pady=5, sticky="nsew")
        self.summary_frame.grid_rowconfigure(0, weight=1)
        self.summary_frame.grid_columnconfigure(0, weight=1)

        # --- Conclusion Frame (Overall Analysis) ---
        self.conclusion_text = tk.Text(self.conclusion_frame, height=15, width=40, state=tk.DISABLED, bg="#f9f9f9", font=('Courier', 9))
        self.conclusion_text.grid(row=0, column=0, padx=5, pady=5, sticky="nsew")
        self.conclusion_frame.grid_rowconfigure(0, weight=1)
        self.conclusion_frame.grid_columnconfigure(0, weight=1)

        # Initial updates
        self.populate_expense_list()
        self.update_summary_gui()
        self.display_overall_conclusion()

        # Save data on closing the window
        master.protocol("WM_DELETE_WINDOW", self.on_closing)

    def on_closing(self):
        """Action to perform when the user closes the window."""
        save_goals()
        save_expenses_csv()
        self.master.destroy()

    def open_calendar(self):
        """Opens a top-level window with a tkcalendar widget."""
        def set_date():
            selected_date = cal.selection_get()
            self.exp_date_var.set(selected_date.strftime('%Y-%m-%d'))
            top.destroy()

        top = Toplevel(self.master)
        top.title("Select Date")

        try:
            initial_date = datetime.strptime(self.exp_date_var.get(), '%Y-%m-%d').date()
        except:
            initial_date = datetime.now().date()

        cal = Calendar(top,
                       selectmode='day',
                       year=initial_date.year,
                       month=initial_date.month,
                       day=initial_date.day,
                       date_pattern='yyyy-mm-dd')
        cal.pack(padx=10, pady=10)

        tk.Button(top, text="Confirm Date", command=set_date, bg="#008CBA", fg="white").pack(pady=5)

    def populate_expense_list(self):
        """Clears and reloads the Listbox with formatted expense entries."""
        self.expense_listbox.delete(0, tk.END)

        # Header for the Listbox
        self.expense_listbox.insert(tk.END, f"{'DATE':<12}{'CATEGORY':<20}{'AMOUNT':>10}")
        self.expense_listbox.itemconfig(0, {'bg': '#e0e0e0', 'fg': '#333333'}) # Highlight header

        # Sort expenses by date descending for listbox display
        sorted_expenses = sorted(EXPENSES, key=lambda x: datetime.strptime(x['date'], '%Y-%m-%d'), reverse=True)

        for expense in sorted_expenses:
            line = f"{expense['date']:<12}{expense['category']:<20}{expense['amount']:>10.2f}"
            self.expense_listbox.insert(tk.END, line)

    def delete_expense_gui(self):
        """Deletes the currently selected expense from the list and data structure."""
        selected_indices = self.expense_listbox.curselection()

        if not selected_indices:
            messagebox.showwarning("Warning", "Please select an expense from the list to delete.")
            return

        # Get the index of the selected item in the listbox (excluding the header at index 0)
        listbox_index = selected_indices[0]

        if listbox_index == 0:
            messagebox.showwarning("Warning", "Cannot delete the header row.")
            return

        try:
            # Recreate the sorted list to map listbox selection back to the EXPENSES list
            sorted_expenses = sorted(EXPENSES, key=lambda x: datetime.strptime(x['date'], '%Y-%m-%d'), reverse=True)
            
            # The actual expense data dictionary is at listbox_index - 1 (because index 0 is the header)
            expense_to_delete = sorted_expenses[listbox_index - 1]

            # Find the exact item in the master EXPENSES list and remove it
            EXPENSES.remove(expense_to_delete)

            messagebox.showinfo("Success", f"Deleted expense of ${expense_to_delete['amount']:,.2f} on {expense_to_delete['date']}.")

            # Update all GUI elements and save data
            self.populate_expense_list()
            self.update_summary_gui()
            self.display_overall_conclusion()
            save_expenses_csv()

        except IndexError:
            messagebox.showerror("Error", "Could not find the selected expense in the database.")
        except ValueError:
             messagebox.showerror("Error", "Error processing list data during deletion.")

    def reset_all_data(self):
        """Clears all expenses, goals, and deletes the data files."""
        if not messagebox.askyesno("Confirm Reset", 
                                   "Are you sure you want to delete ALL financial data (Goals and Expenses)? This cannot be undone."):
            return

        # 1. Clear in-memory data
        FINANCE_GOALS.update({
            'monthly_salary': 0.0,
            'saving_goal': 0.0,
            'max_spending_budget': 0.0
        })
        EXPENSES.clear()

        # 2. Reset GUI variables
        self.salary_var.set(0.0)
        self.saving_var.set(0.0)
        
        # 3. Delete files from disk
        for f in [DATA_FILE, GOAL_FILE]:
            if os.path.exists(f):
                os.remove(f)

        # 4. Update GUI displays
        self.populate_expense_list()
        self.update_summary_gui()
        self.display_overall_conclusion()

        messagebox.showinfo("Success", "All financial data has been reset! You can now set new goals.")


    # --- Core Logic Functions ---

    def set_goals_gui(self):
        """Sets goals using values from GUI entries."""
        try:
            salary = self.salary_var.get()
            saving = self.saving_var.get()
        except tk.TclError:
            messagebox.showerror("Error", "Invalid input. Please enter a valid number for Salary and Saving Goal.")
            return

        if salary <= 0 or saving < 0:
            messagebox.showwarning("Warning", "Salary must be greater than zero, and Saving Goal cannot be negative.")
            return

        if saving > salary:
            messagebox.showwarning("Warning", "Your saving goal exceeds your salary. Please review.")
            return

        FINANCE_GOALS['monthly_salary'] = salary
        FINANCE_GOALS['saving_goal'] = saving
        FINANCE_GOALS['max_spending_budget'] = salary - saving

        messagebox.showinfo("Success", "Goals updated successfully!")
        self.update_summary_gui()
        self.display_overall_conclusion()
        save_goals()

    def add_expense_gui(self):
        """Adds a new expense using values from GUI entries."""
        if FINANCE_GOALS['monthly_salary'] <= 0:
            messagebox.showwarning("Warning", "Please set your Salary and Saving Goal first.")
            return

        try:
            amount = self.exp_amount_var.get()
            date_str = self.exp_date_var.get()
            category = self.exp_category_var.get().strip().title()

            if amount <= 0 or not category:
                messagebox.showwarning("Error", "Amount must be greater than zero and Category cannot be empty.")
                return

            datetime.strptime(date_str, '%Y-%m-%d')

        except tk.TclError:
            messagebox.showerror("Error", "Invalid Amount. Please enter a valid number.")
            return
        except ValueError:
            messagebox.showerror("Error", "Invalid Date format. Please use YYYY-MM-DD.")
            return

        EXPENSES.append({
            'date': date_str,
            'category': category,
            'amount': amount
        })

        self.exp_amount_var.set(0.0) # Clear amount field
        self.exp_category_var.set("") # Clear category field

        messagebox.showinfo("Success", f"Expense of {amount:,.2f} added to '{category}'.")
        self.populate_expense_list()
        self.update_summary_gui()
        self.display_overall_conclusion()
        save_expenses_csv()

    # --- Summary and Conclusion (Unchanged core logic) ---

    def update_summary_gui(self):
        """Updates the current month summary display text widget."""
        salary = FINANCE_GOALS['monthly_salary']
        saving_goal = FINANCE_GOALS['saving_goal']
        spending_budget = FINANCE_GOALS['max_spending_budget']

        self.summary_text.config(state=tk.NORMAL)
        self.summary_text.delete(1.0, tk.END)

        if salary == 0:
            self.summary_text.insert(tk.END, "Goals not set.\nPlease set your Salary and Saving Goal.")
            self.summary_text.config(state=tk.DISABLED)
            return

        current_month = datetime.now().strftime('%Y-%m')
        monthly_expenses = [e for e in EXPENSES if e['date'].startswith(current_month)]

        total_spent = sum(e['amount'] for e in monthly_expenses)
        remaining_budget = spending_budget - total_spent

        summary_output = (
            "--- MONTHLY FINANCIAL GOALS ---\n"
            f"Monthly Salary: {salary:,.2f}\n"
            f"Saving Goal: {saving_goal:,.2f}\n"
            f"Max Spending Budget: {spending_budget:,.2f}\n"
            "\n"
            "--- CURRENT MONTH TRACKING ---\n"
            f"Total Spent: {total_spent:,.2f}\n"
            f"Remaining Budget: {remaining_budget:,.2f}\n"
        )

        if remaining_budget < 0:
            summary_output += "\n🛑 OVER BUDGET! 🛑\n"

        category_totals = defaultdict(float)
        for expense in monthly_expenses:
            category_totals[expense['category']] += expense['amount']

        if category_totals:
            summary_output += "\n--- MONTHLY BREAKDOWN ---\n"
            output_buffer = io.StringIO()
            for category, total in sorted(category_totals.items(), key=lambda item: item[1], reverse=True):
                output_buffer.write(f"- {category:<15}: {total:>8,.2f}\n")

            summary_output += output_buffer.getvalue()

        self.summary_text.insert(tk.END, summary_output)
        self.summary_text.config(state=tk.DISABLED)

    def display_overall_conclusion(self):
        """Analyzes all historical data and updates the conclusion text widget."""
        self.conclusion_text.config(state=tk.NORMAL)
        self.conclusion_text.delete(1.0, tk.END)

        if not EXPENSES:
            self.conclusion_text.insert(tk.END, "No expenses recorded yet. Start logging expenses to see an overall analysis!")
            self.conclusion_text.config(state=tk.DISABLED)
            return

        total_spent_all_time = sum(e['amount'] for e in EXPENSES)
        num_records = len(EXPENSES)

        conclusion_output = (
            "--- ALL-TIME SPENDING ANALYSIS ---\n"
            f"Total Expenses Recorded: {num_records}\n"
            f"Total Amount Spent: {total_spent_all_time:,.2f}\n"
        )

        all_time_category_totals = defaultdict(float)
        for expense in EXPENSES:
            all_time_category_totals[expense['category']] += expense['amount']

        sorted_categories = sorted(all_time_category_totals.items(), key=lambda item: item[1], reverse=True)
        top_categories = sorted_categories[:3]

        if top_categories:
            conclusion_output += "\n--- TOP 3 EXPENSE CATEGORIES ---\n"

            output_buffer = io.StringIO()
            rank = 1
            for category, total in top_categories:
                percentage = (total / total_spent_all_time) * 100 if total_spent_all_time else 0
                output_buffer.write(f"{rank}. {category:<15}: {total:>8,.2f} ({percentage:4.1f}%)\n")
                rank += 1
            conclusion_output += output_buffer.getvalue()

        monthly_spend = defaultdict(float)
        for expense in EXPENSES:
            month_year = expense['date'][:7]
            monthly_spend[month_year] += expense['amount']

        if len(monthly_spend) > 1:
            conclusion_output += "\n--- MONTHLY SPENDING TREND ---\n"
            sorted_months = sorted(monthly_spend.keys())

            output_buffer = io.StringIO()
            for month in sorted_months:
                formatted_month = datetime.strptime(month, '%Y-%m').strftime('%b %Y')
                output_buffer.write(f"{formatted_month:<8}: {monthly_spend[month]:>10,.2f}\n")
            conclusion_output += output_buffer.getvalue()

            latest_months = [monthly_spend[m] for m in sorted_months[-2:]]
            if len(latest_months) == 2:
                change = latest_months[1] - latest_months[0]
                if change > 0:
                    trend_msg = f"Your spending increased by {change:,.2f} in the latest month. Keep an eye on costs!"
                elif change < 0:
                    trend_msg = f"Great job! Your spending decreased by {-change:,.2f} in the latest month."
                else:
                    trend_msg = "Your spending remained stable."
                conclusion_output += f"\nTrend: {trend_msg}\n"

        self.conclusion_text.insert(tk.END, conclusion_output)
        self.conclusion_text.config(state=tk.DISABLED)

# --- MAIN APPLICATION START ---

if __name__ == "__main__":
    root = tk.Tk()
    app = BudgetApp(root)
    root.mainloop()